# सेम्यान्टिक कर्नेललाई OpenBnB MCP सर्भरसँग एकीकृत गर्ने

यो नोटबुकले सेम्यान्टिक कर्नेललाई वास्तविक OpenBnB MCP सर्भरसँग प्रयोग गरेर MCPStdioPlugin मार्फत वास्तविक Airbnb आवास खोज्न कसरी प्रयोग गर्ने भनेर देखाउँछ। LLM पहुँचको लागि, यसले Azure AI Foundry प्रयोग गर्दछ। आफ्नो वातावरण भेरिएबलहरू सेटअप गर्न, तपाईं [सेटअप पाठ](/00-course-setup/README.md) अनुसरण गर्न सक्नुहुन्छ।


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP प्लगइन कनेक्शन बनाउने

हामी [OpenBnB MCP server](https://github.com/openbnb-org/mcp-server-airbnb) सँग MCPStdioPlugin प्रयोग गरेर जडान गर्नेछौं। यो सर्भरले @openbnb/mcp-server-airbnb प्याकेज मार्फत Airbnb खोजी सुविधा प्रदान गर्दछ।


## क्लाइन्ट बनाउँदै

यस नमुनामा, हामी हाम्रो LLM पहुँचको लागि Azure AI Foundry प्रयोग गर्नेछौं। सुनिश्चित गर्नुहोस् कि तपाईंको वातावरण भेरिएबलहरू सही रूपमा सेट गरिएको छ।


## वातावरण कन्फिगरेसन

Azure OpenAI सेटिङहरू कन्फिगर गर्नुहोस्। सुनिश्चित गर्नुहोस् कि तपाईंसँग निम्न वातावरण भेरिएबलहरू सेट गरिएको छ:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP एकीकरण बुझ्दै

यो नोटबुकले **वास्तविक OpenBnB MCP सर्भर** सँग जडान गर्दछ, जसले वास्तविक Airbnb खोजी कार्यक्षमता प्रदान गर्दछ।

### यो कसरी काम गर्छ:

1. **MCPStdioPlugin**: MCP सर्भरसँग मानक इनपुट/आउटपुट मार्फत सञ्चार गर्दछ  
2. **वास्तविक NPM प्याकेज**: `@openbnb/mcp-server-airbnb` लाई npx मार्फत डाउनलोड र चलाउँछ  
3. **प्रत्यक्ष डाटा**: Airbnb को API बाट वास्तविक सम्पत्तिको डाटा फिर्ता गर्दछ  
4. **फङ्सन पत्ता लगाउने**: एजेन्टले MCP सर्भरबाट उपलब्ध फङ्सनहरू स्वचालित रूपमा पत्ता लगाउँछ  

### उपलब्ध फङ्सनहरू:

OpenBnB MCP सर्भरले सामान्यतया निम्न फङ्सनहरू प्रदान गर्दछ:  
- **search_listings** - स्थान र मापदण्डद्वारा Airbnb सम्पत्तिहरू खोज्नुहोस्  
- **get_listing_details** - विशेष सम्पत्तिहरूको विस्तृत जानकारी प्राप्त गर्नुहोस्  
- **check_availability** - विशेष मितिहरूको लागि उपलब्धता जाँच गर्नुहोस्  
- **get_reviews** - सम्पत्तिहरूको लागि समीक्षाहरू प्राप्त गर्नुहोस्  
- **get_host_info** - सम्पत्तिका होस्टहरूको जानकारी प्राप्त गर्नुहोस्  

### पूर्वआवश्यकताहरू:

- तपाईंको प्रणालीमा **Node.js** स्थापना भएको हुनुपर्छ  
- MCP सर्भर प्याकेज डाउनलोड गर्नको लागि **इन्टरनेट जडान** आवश्यक छ  
- **NPX** उपलब्ध हुनुपर्छ (यो Node.js सँगै आउँछ)  

### जडान परीक्षण गर्दै:

तपाईं MCP सर्भरलाई म्यानुअली परीक्षण गर्न निम्न कमाण्ड चलाउन सक्नुहुन्छ:  
```bash
npx -y @openbnb/mcp-server-airbnb
```  

यसले OpenBnB MCP सर्भर डाउनलोड र सुरु गर्नेछ, जससँग Semantic Kernel ले वास्तविक Airbnb डाटाको लागि जडान गर्दछ।  


## OpenBnB MCP सर्भरसँग एजेन्ट चलाउने

अब हामी OpenBnB MCP सर्भरसँग जडान हुने AI एजेन्ट चलाउनेछौं, जसले स्टकहोममा २ वयस्क र १ बच्चाका लागि वास्तविक Airbnb आवास खोज्छ। खोज मापदण्ड परिवर्तन गर्न `user_inputs` सूचीलाई परिमार्जन गर्न नहिचकिचाउनुहोस्।


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

सारांश  
बधाई छ! तपाईंले सफलतापूर्वक एउटा AI एजेन्ट निर्माण गर्नुभएको छ जसले वास्तविक संसारको आवास खोजसँग Model Context Protocol (MCP) प्रयोग गरेर एकीकृत गर्दछ:

प्रयोग गरिएका प्रविधिहरू:  
- Semantic Kernel - Azure OpenAI प्रयोग गरेर बुद्धिमान एजेन्ट निर्माण गर्न  
- Azure AI Foundry - LLM क्षमता र च्याट पूरा गर्न  
- MCP (Model Context Protocol) - मानकीकृत उपकरण एकीकरणका लागि  
- OpenBnB MCP Server - वास्तविक Airbnb खोज कार्यक्षमताका लागि  
- Node.js/NPX - बाह्य MCP सर्भर चलाउनका लागि  

तपाईंले के सिक्नुभयो:  
- MCP एकीकरण: Semantic Kernel एजेन्टलाई बाह्य MCP सर्भरसँग जडान गर्ने  
- वास्तविक-समय डेटा पहुँच: प्रत्यक्ष API मार्फत वास्तविक Airbnb सम्पत्तिहरू खोज्ने  
- प्रोटोकल सञ्चार: एजेन्ट र MCP सर्भर बीच stdio सञ्चार प्रयोग गर्ने  
- फङ्सन पत्ता लगाउने: MCP सर्भरहरूबाट उपलब्ध फङ्सनहरू स्वचालित रूपमा पत्ता लगाउने  
- स्ट्रिमिङ प्रतिक्रियाहरू: वास्तविक-समयमा फङ्सन कलहरू समात्ने र लग गर्ने  
- HTML रेन्डरिङ: एजेन्ट प्रतिक्रियाहरूलाई शैलीबद्ध तालिका र अन्तरक्रियात्मक प्रदर्शनहरूमा ढाल्ने  

अर्को चरणहरू:  
- थप MCP सर्भरहरू एकीकृत गर्नुहोस् (मौसम, उडान, रेस्टुरेन्ट)  
- MCP र A2A प्रोटोकलहरू संयोजन गरेर बहु-एजेन्ट प्रणाली निर्माण गर्नुहोस्  
- आफ्नै डेटा स्रोतहरूको लागि कस्टम MCP सर्भरहरू सिर्जना गर्नुहोस्  
- सत्रहरूमा स्थायी संवाद स्मृति कार्यान्वयन गर्नुहोस्  
- MCP सर्भर व्यवस्थापनसहित एजेन्टलाई Azure Functions मा तैनाथ गर्नुहोस्  
- प्रयोगकर्ता प्रमाणीकरण र बुकिङ क्षमताहरू थप्नुहोस्  

MCP आर्किटेक्चरका प्रमुख फाइदाहरू:  
- मानकीकरण: AI एजेन्टलाई बाह्य उपकरणहरूसँग जडान गर्नको लागि सार्वभौमिक प्रोटोकल  
- वास्तविक-समय डेटा: विभिन्न सेवाहरूबाट प्रत्यक्ष, अद्यावधिक जानकारीको पहुँच  
- विस्तारयोग्यता: नयाँ डेटा स्रोतहरू र उपकरणहरूको सजिलो एकीकरण  
- अन्तरसञ्चालन क्षमता: विभिन्न AI फ्रेमवर्कहरू र एजेन्ट प्लेटफर्महरूमा काम गर्ने  
- चिन्ताको छुट्टैकरण: AI तर्क र बाह्य डेटा पहुँच बीच स्पष्ट भिन्नता



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादहरूमा त्रुटि वा अशुद्धता हुन सक्छ। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्त्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।
